In [1]:
import pyomo.environ as pyo
from pyomo.core.base.set import UnindexedComponent_set
#from farmer_example_block_robby import build_block_model
from pyomo.dae.flatten import flatten_components_along_sets
from pyomo.environ import *
from pyomo.dae import *
import numpy as np

## Analytical problem with block

In [2]:
#time_an = list(np.linspace(0,1,9))
def create_model(CA_init=5, T_init=300):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    # time points 
    timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    
    m= ConcreteModel()

    m.CA0 = Var(initialize=CA_init, bounds=(1.0,5.0),within=NonNegativeReals )
    m.T = Var(initialize=T_init, bounds=(300,700),within=NonNegativeReals)
    
    m.CA0.fix()
    m.T.fix()
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=timepoint)
    '''
    m.A1 = Var(initialize = theta_p['A1'])
    m.A2 = Var(initialize = theta_p['A2'])
    m.E1 = Var(initialize = theta_p['E1'])
    m.E2 = Var(initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
    
    def k1_rule(m):
        return m.k1 == m.A1*exp(-m.E1*1000/m.R/m.T)

    def k2_rule(m):
        return m.k2 == m.A2 * exp(-m.E2*1000/m.R/m.T)

    m.k1 = Var(initialize=0.2, within=NonNegativeReals) # 1/hr
    m.k2 = Var(initialize=0.1, within=NonNegativeReals)
    
    def conc_CA(m,t):
        return m.CA[t] == m.CA0*exp(-m.k1*t)
    def conc_CB(m,t):
        return m.CB[t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
    def conc_CC(m,t):
        return m.CC[t] == m.CA0 - m.CA0*exp(-m.k1*t) - m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))

    m.k1_r = Constraint(rule=k1_rule)
    m.k2_r = Constraint(rule=k2_rule)
    m.rate_CA = Constraint(m.t, rule=conc_CA)
    m.rate_CB = Constraint(m.t, rule=conc_CB)
    m.rate_CC = Constraint(m.t, rule=conc_CC)
    '''
    
    #m.Obj = Objective(rule=0, sense=maximize)
        
    return m

In [3]:
mod_simp = create_model(CA_init=5, T_init=300)

#solver = SolverFactory('ipopt')
#solver.options['halt_on_ampl_error'] = 'yes'
#solver.solve(mod_simp, tee=True)

In [4]:
def construct_block(m):
    
    m.scena = Set(initialize=list(range(2)))
    theta_pa = [{'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}, 
                {'A1': 84.79085853498033*1.01, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}]
    
    def block_build(b, s):
        b.A1 = Var(initialize = theta_pa[s]['A1'])
        b.A2 = Var(initialize = theta_pa[s]['A2'])
        b.E1 = Var(initialize = theta_pa[s]['E1'])
        b.E2 = Var(initialize = theta_pa[s]['E2'])
        
        b.A1.fix()
        b.A2.fix()
        b.E1.fix()
        b.E2.fix()
        
        b.k1 = Var(initialize=0.2, within=NonNegativeReals) # 1/hr
        b.k2 = Var(initialize=0.1, within=NonNegativeReals)
        
        b.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
        
    m.lsb = Block(m.scena, rule=block_build)
    
    
    def k1_rule_block(m,s):
        return m.lsb[s].k1 == m.lsb[s].A1*exp(-m.lsb[s].E1*1000/m.R/m.T)

    def k2_rule_block(m,s):
        return m.lsb[s].k2 == m.lsb[s].A2 * exp(-m.lsb[s].E2*1000/m.R/m.T)
    
    def conc_CA_block(m,s,t):
        return m.lsb[s].CA[t] == m.CA0*exp(-m.lsb[s].k1*t)
    def conc_CB_block(m,s,t):
        return m.lsb[s].CB[t] == m.lsb[s].k1*m.CA0/(m.lsb[s].k2-m.lsb[s].k1) * (exp(-m.lsb[s].k1*t) - exp(-m.lsb[s].k2*t))
    def conc_CC_block(m,s,t):
        return m.lsb[s].CC[t] == m.CA0 - m.CA0*exp(-m.lsb[s].k1*t) - m.lsb[s].k1*m.CA0/(m.lsb[s].k2-m.lsb[s].k1) * (exp(-m.lsb[s].k1*t) - exp(-m.lsb[s].k2*t))

    m.k1_r_block = Constraint(m.scena,rule=k1_rule_block)
    m.k2_r_block = Constraint(m.scena,rule=k2_rule_block)
    m.rate_CA_block = Constraint(m.scena,m.t, rule=conc_CA_block)
    m.rate_CB_block = Constraint(m.scena,m.t, rule=conc_CB_block)
    m.rate_CC_block = Constraint(m.scena,m.t, rule=conc_CC_block)
    
    return m

In [5]:
mod_simp_bl = construct_block(mod_simp)

#solver = SolverFactory('ipopt')
#solver.options['halt_on_ampl_error'] = 'yes'
#solver.solve(mod_simp, tee=True)

In [7]:
mod_simp_bl, mod_flatten = all_sets(mod_simp_bl)


i is: 0
sets is: (<pyomo.core.base.global_set._UnindexedComponent_set object at 0x1170d3070>,)
vars is: [<pyomo.core.base.var.ScalarVar object at 0x17fba4cf0>, <pyomo.core.base.var.ScalarVar object at 0x17fba4c80>]
i is: 1
sets is: (<pyomo.core.base.set.OrderedScalarSet object at 0x17fbc50b0>,)
vars is: [<pyomo.core.base.var.IndexedVar object at 0x17fc0cdf0>, <pyomo.core.base.var.IndexedVar object at 0x17fc07910>, <pyomo.core.base.var.IndexedVar object at 0x17fc07e80>, <pyomo.core.base.var.IndexedVar object at 0x17fc07fa0>, <pyomo.core.base.var.IndexedVar object at 0x17fc078b0>, <pyomo.core.base.var.IndexedVar object at 0x17fc07a60>]
i is: 2
sets is: (<pyomo.core.base.set.OrderedScalarSet object at 0x17fbc50b0>, <pyomo.core.base.set.OrderedScalarSet object at 0x17fba4eb0>)
vars is: [<pyomo.core.base.var.IndexedVar object at 0x17fc079a0>, <pyomo.core.base.var.IndexedVar object at 0x17fc07af0>, <pyomo.core.base.var.IndexedVar object at 0x17fc07b80>]
Ipopt 3.13.3: 

**********************

In [10]:
display(mod_flatten)

Model unknown

  Variables:
    unindexed_vars : Size=2, Index=unindexed_vars_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   1.0 :     5 :   5.0 :  True :  True : NonNegativeReals
          1 :   300 :   300 :   700 :  True :  True : NonNegativeReals
    var_1_0 : Size=2, Index=scena
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :  None : 84.79085853498033 :  None :  True :  True :  Reals
          1 :  None : 85.63876712033013 :  None :  True :  True :  Reals
    var_1_1 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :  None : 371.71773413976416 :  None :  True :  True :  Reals
          1 :  None : 371.71773413976416 :  None :  True :  True :  Reals
    var_1_2 : Size=2, Index=scena
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :  None : 7.777032028026428 :  None :  True :  True :  Reals
          1 :  None : 7.

In [6]:
def all_sets(m):

    # It makes sense to build the flattened model ahead of time
    # in this case, so we don't need to know what "set combinations"
    # we're looking for a priori
    flattened_model = pyo.ConcreteModel()

    sets = tuple(m.component_data_objects(pyo.Set))
    # This partitions model components according to how they are indexed
    sets_list, vars_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Var,
    )
    for i, (sets, vars) in enumerate(zip(sets_list, vars_list)):
        print('i is:', i)
        print('sets is:', sets)
        print('vars is:', vars)
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_vars = pyo.Reference(vars)
        else:
            for j, var in enumerate(vars):
                flattened_model.add_component("var_%s_%s" % (i, j), var)

    sets_list, cons_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Constraint,
    )
    for i, (sets, cons) in enumerate(zip(sets_list, cons_list)):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_constraints = pyo.Reference(cons)
        else:
            for j, con in enumerate(cons):
                flattened_model.add_component("con_%s_%s" % (i, j), con)

    #flattened_model.obj = pyo.Reference(m.OBJ)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(flattened_model, tee=True)
    
    return m, flattened_model



## Analytical problem, with indexed T

In [12]:
#time_an = list(np.linspace(0,1,9))
def create_model2(CA_init=5, T_init=300):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    # time points 
    timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    
    m= ConcreteModel()
    
    m.t = Set(initialize=timepoint)

    m.CA0 = Var(initialize=CA_init, bounds=(1.0,5.0),within=NonNegativeReals )
    m.T = Var(m.t, initialize=T_init, bounds=(300,700),within=NonNegativeReals)
    
    m.CA0.fix()
    m.T.fix()
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=timepoint)
    '''
    m.A1 = Var(initialize = theta_p['A1'])
    m.A2 = Var(initialize = theta_p['A2'])
    m.E1 = Var(initialize = theta_p['E1'])
    m.E2 = Var(initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
    
    def k1_rule(m):
        return m.k1 == m.A1*exp(-m.E1*1000/m.R/m.T)

    def k2_rule(m):
        return m.k2 == m.A2 * exp(-m.E2*1000/m.R/m.T)

    m.k1 = Var(initialize=0.2, within=NonNegativeReals) # 1/hr
    m.k2 = Var(initialize=0.1, within=NonNegativeReals)
    
    def conc_CA(m,t):
        return m.CA[t] == m.CA0*exp(-m.k1*t)
    def conc_CB(m,t):
        return m.CB[t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
    def conc_CC(m,t):
        return m.CC[t] == m.CA0 - m.CA0*exp(-m.k1*t) - m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))

    m.k1_r = Constraint(rule=k1_rule)
    m.k2_r = Constraint(rule=k2_rule)
    m.rate_CA = Constraint(m.t, rule=conc_CA)
    m.rate_CB = Constraint(m.t, rule=conc_CB)
    m.rate_CC = Constraint(m.t, rule=conc_CC)
    '''
    
    #m.Obj = Objective(rule=0, sense=maximize)
        
    return m

In [13]:
def construct_block2(m):
    
    m.scena = Set(initialize=list(range(2)))
    theta_pa = [{'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}, 
                {'A1': 84.79085853498033*1.01, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}]
    
    def block_build(b, s):
        b.A1 = Var(initialize = theta_pa[s]['A1'])
        b.A2 = Var(initialize = theta_pa[s]['A2'])
        b.E1 = Var(initialize = theta_pa[s]['E1'])
        b.E2 = Var(initialize = theta_pa[s]['E2'])
        
        b.A1.fix()
        b.A2.fix()
        b.E1.fix()
        b.E2.fix()
        
        b.k1 = Var(m.t, initialize=0.2, within=NonNegativeReals) # 1/hr
        b.k2 = Var(m.t, initialize=0.1, within=NonNegativeReals)
        
        b.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
        
    m.lsb = Block(m.scena, rule=block_build)
    
    
    def k1_rule_block(m,s,t):
        return m.lsb[s].k1[t] == m.lsb[s].A1*exp(-m.lsb[s].E1*1000/m.R/m.T[t])

    def k2_rule_block(m,s,t):
        return m.lsb[s].k2[t] == m.lsb[s].A2 * exp(-m.lsb[s].E2*1000/m.R/m.T[t])
    
    def conc_CA_block(m,s,t):
        return m.lsb[s].CA[t] == m.CA0*exp(-m.lsb[s].k1[t]*t)
    def conc_CB_block(m,s,t):
        return m.lsb[s].CB[t] == m.lsb[s].k1[t]*m.CA0/(m.lsb[s].k2[t]-m.lsb[s].k1[t]) * (exp(-m.lsb[s].k1[t]*t) - exp(-m.lsb[s].k2[t]*t))
    def conc_CC_block(m,s,t):
        return m.lsb[s].CC[t] == m.CA0 - m.CA0*exp(-m.lsb[s].k1[t]*t) - m.lsb[s].k1[t]*m.CA0/(m.lsb[s].k2[t]-m.lsb[s].k1[t]) * (exp(-m.lsb[s].k1[t]*t) - exp(-m.lsb[s].k2[t]*t))

    m.k1_r_block = Constraint(m.scena,m.t,rule=k1_rule_block)
    m.k2_r_block = Constraint(m.scena,m.t,rule=k2_rule_block)
    m.rate_CA_block = Constraint(m.scena,m.t, rule=conc_CA_block)
    m.rate_CB_block = Constraint(m.scena,m.t, rule=conc_CB_block)
    m.rate_CC_block = Constraint(m.scena,m.t, rule=conc_CC_block)
    
    return m

In [17]:
model2 = create_model2()
model2_block = construct_block2(model2)
model2_block, model2_flatten = all_sets(model2_block)

    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().
i is: 0
sets is: (<pyomo.core.base.global_set._UnindexedComponent_set object at 0x1170d3070>,)
vars is: [<pyomo.core.base.var.ScalarVar object at 0x17fce9b30>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5c940>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5c9a0>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5ca00>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5ca60>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5cac0>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5cb20>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5cb80>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5cbe0>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5cc40>]
i is: 1


In [18]:
display(model2_flatten)

Model unknown

  Variables:
    unindexed_vars : Size=10, Index=unindexed_vars_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   1.0 :     5 :   5.0 :  True :  True : NonNegativeReals
          1 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          2 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          3 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          4 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          5 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          6 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          7 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          8 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          9 :   300 :   300 :   700 :  True :  True : NonNegativeReals
    var_1_0 : Size=2, Index=scena
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :  None : 84.790858534980

## Pyomo DAE problem 

In [26]:
#time_an = list(np.linspace(0,1,9))
def create_model_dae(CA_init=5, T_init=300):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    # time points 
    timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    
    m= ConcreteModel()
    
    m.t = ContinuousSet(bounds = (0.0,1))
    
    m.t_con = Set(initialize=timepoint)

    m.CA0 = Var(initialize=CA_init, bounds=(1.0,5.0),within=NonNegativeReals )
    m.T = Var(m.t, initialize=T_init, bounds=(300,700),within=NonNegativeReals)
    
    m.CA0.fix()
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    '''
    m.A1 = Var(initialize = theta_p['A1'])
    m.A2 = Var(initialize = theta_p['A2'])
    m.E1 = Var(initialize = theta_p['E1'])
    m.E2 = Var(initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
    m.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
    
    def k1_rule(m):
        return m.k1 == m.A1*exp(-m.E1*1000/m.R/m.T)

    def k2_rule(m):
        return m.k2 == m.A2 * exp(-m.E2*1000/m.R/m.T)

    m.k1 = Var(initialize=0.2, within=NonNegativeReals) # 1/hr
    m.k2 = Var(initialize=0.1, within=NonNegativeReals)
    
    def conc_CA(m,t):
        return m.CA[t] == m.CA0*exp(-m.k1*t)
    def conc_CB(m,t):
        return m.CB[t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
    def conc_CC(m,t):
        return m.CC[t] == m.CA0 - m.CA0*exp(-m.k1*t) - m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))

    m.k1_r = Constraint(rule=k1_rule)
    m.k2_r = Constraint(rule=k2_rule)
    m.rate_CA = Constraint(m.t, rule=conc_CA)
    m.rate_CB = Constraint(m.t, rule=conc_CB)
    m.rate_CC = Constraint(m.t, rule=conc_CC)
    '''
    
    #m.Obj = Objective(rule=0, sense=maximize)
        
    return m

In [27]:
def construct_block_dae(m):
    
    m.scena = Set(initialize=list(range(2)))
    theta_pa = [{'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}, 
                {'A1': 84.79085853498033*1.01, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}]
    
    def block_build(b, s):
        b.A1 = Var(initialize = theta_pa[s]['A1'])
        b.A2 = Var(initialize = theta_pa[s]['A2'])
        b.E1 = Var(initialize = theta_pa[s]['E1'])
        b.E2 = Var(initialize = theta_pa[s]['E2'])
        
        b.A1.fix()
        b.A2.fix()
        b.E1.fix()
        b.E2.fix()
        
        b.k1 = Var(m.t, initialize=0.2, within=NonNegativeReals) # 1/hr
        b.k2 = Var(m.t, initialize=0.1, within=NonNegativeReals)
        
        b.CA = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CB = Var(m.t, initialize=0.5, within=NonNegativeReals)
        b.CC = Var(m.t, initialize=0.5, within=NonNegativeReals)
        
        b.dCAdt = DerivativeVar(b.CA, wrt=m.t)
        b.dCBdt = DerivativeVar(b.CB, wrt=m.t)
        b.dCCdt = DerivativeVar(b.CC, wrt=m.t)
        
    m.lsb = Block(m.scena, rule=block_build)
    
    
    def k1_rule_block(m,s,t):
        return m.lsb[s].k1[t] == m.lsb[s].A1*exp(-m.lsb[s].E1*1000/m.R/m.T[t])

    def k2_rule_block(m,s,t):
        return m.lsb[s].k2[t] == m.lsb[s].A2 * exp(-m.lsb[s].E2*1000/m.R/m.T[t])
    
    def conc_CA_block(m,s,t):
        return m.lsb[s].dCAdt[t] == -m.lsb[s].k1[t]*m.lsb[s].CA[t]
    def conc_CB_block(m,s,t):
        return m.lsb[s].dCBdt[t] == m.lsb[s].k1[t]*m.lsb[s].CA[t] - m.lsb[s].k2[t]*m.lsb[s].CB[t]
    def conc_CC_block(m,s,t):
        return m.lsb[s].CC[t] + m.lsb[s].CB[t] + m.lsb[s].CA[t] == m.CA0
    
    m.k1_r_block = Constraint(m.scena,m.t,rule=k1_rule_block)
    m.k2_r_block = Constraint(m.scena,m.t,rule=k2_rule_block)
    m.rate_CA_block = Constraint(m.scena,m.t, rule=conc_CA_block)
    m.rate_CB_block = Constraint(m.scena,m.t, rule=conc_CB_block)
    m.rate_CC_block = Constraint(m.scena,m.t, rule=conc_CC_block)
    
    
    for s in m.scena:
        m.lsb[s].CB[0.0].fix(0.0)
        m.lsb[s].CC[0.0].fix(0.0)
    return m

In [28]:
def discretizer(m, NFE=32):
    discretizer = TransformationFactory('dae.collocation')
    discretizer.apply_to(m, nfe=NFE, ncp=3, wrt=m.t)
    return m 

In [29]:
model2 = create_model_dae()
model2_block = construct_block_dae(model2)
model2_block = discretizer(model2_block)
model2_block.T.fix()

model2_block, model2_flatten = all_sets(model2_block)

i is: 0
sets is: (<pyomo.core.base.global_set._UnindexedComponent_set object at 0x1170d3070>,)
vars is: [<pyomo.core.base.var.ScalarVar object at 0x17fdd9cf0>, <pyomo.core.base.var._GeneralVarData object at 0x17fdd22e0>, <pyomo.core.base.var._GeneralVarData object at 0x17fce43a0>, <pyomo.core.base.var._GeneralVarData object at 0x1801f9820>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5c4c0>, <pyomo.core.base.var._GeneralVarData object at 0x1801f8400>, <pyomo.core.base.var._GeneralVarData object at 0x1801f9880>, <pyomo.core.base.var._GeneralVarData object at 0x17fd5c460>, <pyomo.core.base.var._GeneralVarData object at 0x1801f9c40>, <pyomo.core.base.var._GeneralVarData object at 0x1801f98e0>, <pyomo.core.base.var._GeneralVarData object at 0x1801f1b80>, <pyomo.core.base.var._GeneralVarData object at 0x1801f9ca0>, <pyomo.core.base.var._GeneralVarData object at 0x1801f9940>, <pyomo.core.base.var._GeneralVarData object at 0x1801f1dc0>, <pyomo.core.base.var._GeneralVarData object at 

Ipopt 3.13.3: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     5382
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      386

Total number of variables............................:     1546
                     variables with only lower bounds:      966
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1546
Total number of inequ

In [30]:
display(model2_flatten)

Model unknown

  Variables:
    unindexed_vars : Size=98, Index=unindexed_vars_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   1.0 :     5 :   5.0 :  True :  True : NonNegativeReals
          1 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          2 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          3 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          4 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          5 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          6 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          7 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          8 :   300 :   300 :   700 :  True :  True : NonNegativeReals
          9 :   300 :   300 :   700 :  True :  True : NonNegativeReals
         10 :   300 :   300 :   700 :  True :  True : NonNegativeReals
         11 :   300 :   300 :   700 :  True :  True : NonNegativeReals
  

        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 3.8835587369015387 :  None : False : False : NonNegativeReals
          1 :     0 : 3.8737579397197646 :  None : False : False : NonNegativeReals
    var_1_206 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 :  3.666764157894052 :  None : False : False : NonNegativeReals
          1 :     0 : 3.6554100802415714 :  None : False : False : NonNegativeReals
    var_1_207 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 3.5172482565328744 :  None : False : False : NonNegativeReals
          1 :     0 :  3.504897755082722 :  None : False : False : NonNegativeReals
    var_1_208 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 3.4538824364370724 :  None : False : False : NonNegativeReals
        

    var_1_436 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 0.9190946987382809 :  None : False : False : NonNegativeReals
          1 :     0 : 0.9241869799194712 :  None : False : False : NonNegativeReals
    var_1_437 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 0.9506209645422351 :  None : False : False : NonNegativeReals
          1 :     0 : 0.9558122974766569 :  None : False : False : NonNegativeReals
    var_1_438 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 0.9644028121406194 :  None : False : False : NonNegativeReals
          1 :     0 : 0.9696362526025889 :  None : False : False : NonNegativeReals
    var_1_439 : Size=2, Index=scena
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 1.0079870045071397 :  None : False

        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :  None :  -1.645701854921677 :  None : False : False :  Reals
          1 :  None : -1.6647983815754712 :  None : False : False :  Reals
    var_1_664 : Size=2, Index=scena
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :  None : -1.6671304800499953 :  None : False : False :  Reals
          1 :  None : -1.6856504809656156 :  None : False : False :  Reals
    var_1_665 : Size=2, Index=scena
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :  None : -1.6759358505947377 :  None : False : False :  Reals
          1 :  None : -1.6942061342978794 :  None : False : False :  Reals
    var_1_666 : Size=2, Index=scena
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :  None : -1.7016462628611562 :  None : False : False :  Reals
          1 :  None :  -1.719136374343554 :  None : False : False :

          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_87 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 : -1.7763568394002505e-15 :   0.0
          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_88 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 : -1.7763568394002505e-15 :   0.0
          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_89 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 : -1.7763568394002505e-15 :   0.0
          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_90 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 : -1.7763568394002505e-15 :   0.0
          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_91 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 : -1.7763568394002505e-15 :   0.0
          1 :   0.0 : -1.7763568394002505e-15 :   0.0
    con_0_92 : Size=2
    

    con_0_398 : Size=2
        Key : Lower : Body : Upper
          0 :   0.0 :  0.0 :   0.0
          1 :   0.0 :  0.0 :   0.0
    con_0_399 : Size=2
        Key : Lower : Body : Upper
          0 :   0.0 :  0.0 :   0.0
          1 :   0.0 :  0.0 :   0.0
    con_0_400 : Size=2
        Key : Lower : Body                  : Upper
          0 :   0.0 :                   0.0 :   0.0
          1 :   0.0 : 8.881784197001252e-16 :   0.0
    con_0_401 : Size=2
        Key : Lower : Body : Upper
          0 :   0.0 :  0.0 :   0.0
          1 :   0.0 :  0.0 :   0.0
    con_0_402 : Size=2
        Key : Lower : Body : Upper
          0 :   0.0 :  0.0 :   0.0
          1 :   0.0 :  0.0 :   0.0
    con_0_403 : Size=2
        Key : Lower : Body : Upper
          0 :   0.0 :  0.0 :   0.0
          1 :   0.0 :  0.0 :   0.0
    con_0_404 : Size=2
        Key : Lower : Body                  : Upper
          0 :   0.0 : 8.881784197001252e-16 :   0.0
          1 :   0.0 :                   0.0 :   0.0
  

    con_0_753 : Size=2
        Key : Lower : Body                   : Upper
          0 :   0.0 : -5.773159728050814e-15 :   0.0
          1 :   0.0 : -3.552713678800501e-15 :   0.0
    con_0_754 : Size=2
        Key : Lower : Body                    : Upper
          0 :   0.0 :  -6.217248937900877e-14 :   0.0
          1 :   0.0 : -4.4853010194856324e-14 :   0.0
    con_0_755 : Size=2
        Key : Lower : Body                  : Upper
          0 :   0.0 : 3.197442310920451e-14 :   0.0
          1 :   0.0 : 3.863576125695545e-14 :   0.0
    con_0_756 : Size=2
        Key : Lower : Body                   : Upper
          0 :   0.0 :  4.440892098500626e-15 :   0.0
          1 :   0.0 : -3.907985046680551e-14 :   0.0
    con_0_757 : Size=2
        Key : Lower : Body                   : Upper
          0 :   0.0 : -5.728750807065808e-14 :   0.0
          1 :   0.0 :  4.796163466380676e-14 :   0.0
    con_0_758 : Size=2
        Key : Lower : Body                   : Upper
          0 : 